In [1]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import numpy as np
from tensorflow.keras.preprocessing import image_dataset_from_directory

import warnings
warnings.filterwarnings("ignore")

2024-09-24 12:48:25.670021: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-24 12:48:25.687597: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-24 12:48:25.693109: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-24 12:48:25.705566: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-24 12:48:26.722204: W tensorflow/compiler/tf2

In [2]:
# Load dataset from directory
train_dir = 'datasets/train'
test_dir = 'datasets/test'

# Load train and test datasets
train_dataset = image_dataset_from_directory(
    train_dir,
    image_size=(224, 224),
    batch_size=32,
    label_mode='int', 
    shuffle=True,
    seed=123 
)

test_dataset = image_dataset_from_directory(
    test_dir,
    image_size=(224, 224),
    batch_size=32,
    label_mode='int',
    shuffle=False, 
    seed=123
)

Found 1034 files belonging to 3 classes.


I0000 00:00:1727178508.962213   40259 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1727178508.998966   40259 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1727178509.003502   40259 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1727178509.008525   40259 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

Found 128 files belonging to 3 classes.


In [3]:
# Function to extract features from the images using CNN
def extract_features(dataset, cnn_model):
    features = []
    labels = []

    for images, lbls in dataset:
        try:
            feature_batch = cnn_model.predict(images)
            features.append(feature_batch)
            labels.append(lbls.numpy()) # Convert labels to numpy arrays
        except Exception as e:
            print("Error processing a batch:", e)
            continue

    features = np.vstack(features) # Shape to (samples, features)
    labels = np.concatenate(labels) # Combine all label batches
    return features, labels

In [4]:
# Load pre-trained CNN model (VGG16 without top layer)
cnn_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [5]:
# Extract features from train and test sets
train_features, train_labels = extract_features(train_dataset, cnn_model)
test_features, test_labels = extract_features(test_dataset, cnn_model)

I0000 00:00:1727178515.290313   40378 service.cc:146] XLA service 0x7f2fbc004830 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1727178515.290331   40378 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce GTX 1660 Ti, Compute Capability 7.5
2024-09-24 12:48:35.297523: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-09-24 12:48:35.362396: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907
2024-09-24 12:48:36.474351: W external/local_tsl/tsl/framework/bfc_allocator.cc:291] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.04GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-09-24 12:48:37.318529: W external/local_tsl/tsl/framework/bfc_allocator.cc:291]

1/1 ━━━━━━━━━━━━━━━━━━━━ 10s 10s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step


I0000 00:00:1727178524.988483   40378 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━

2024-09-24 12:48:55.640571: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step


2024-09-24 12:48:56.435189: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [6]:
# Flatten the features
train_features = train_features.reshape(train_features.shape[0], -1)
test_features = test_features.reshape(test_features.shape[0], -1)

In [7]:
# Normalize the features for SVM
scaler = StandardScaler()
train_features_scaled = scaler.fit_transform(train_features)
test_features_scaled = scaler.transform(test_features)

In [47]:
# SVM model
svm_model = SVC(kernel='linear')
svm_model.fit(train_features_scaled, train_labels)

# Predict and evaluate
test_predictions = svm_model.predict(test_features_scaled)
train_preditions = svm_model.predict(train_features_scaled)
test_accuracy = accuracy_score(test_labels, test_predictions)
train_accuracy = accuracy_score(train_labels, train_preditions)
print(f"Test Accuracy: {test_accuracy:.3f}")
print("Train accuracy: {:.3f}".format(train_accuracy))

Test Accuracy: 0.898
Train accuracy: 1.000


In [50]:
import pickle

with open("models/vgg16_svm_ensemble.pkl", "wb") as file:
    pickle.dump(svm_model, file)

In [ ]:
import pickle
from sklearn.metrics import accuracy_score, classification_report # type: ignore

In [9]:
with open("models/vgg16_svm_ensemble.pkl", 'rb') as file:
    model = pickle.load(file)

In [10]:
class_names = ["angular_leaf_spot", "bean_rust", "healthy"]
test_pred = model.predict(test_features)

In [11]:
report = classification_report(test_labels, test_pred, target_names=class_names)
print(report)

                   precision    recall  f1-score   support

angular_leaf_spot       0.95      0.84      0.89        43
        bean_rust       0.88      0.70      0.78        43
          healthy       0.75      1.00      0.86        42

         accuracy                           0.84       128
        macro avg       0.86      0.84      0.84       128
     weighted avg       0.86      0.84      0.84       128

